<div align="center">
<a href="https://rapidfire.ai/"><img src="https://raw.githubusercontent.com/RapidFireAI/rapidfireai/main/docs/images/RapidFire - Blue bug -white text.svg" width="115"></a>
<a href="https://discord.gg/6vSTtncKNN"><img src="https://raw.githubusercontent.com/RapidFireAI/rapidfireai/main/docs/images/discord-button.svg" width="145"></a>
<a href="https://oss-docs.rapidfire.ai/"><img src="https://raw.githubusercontent.com/RapidFireAI/rapidfireai/main/docs/images/documentation-button.svg" width="125"></a>
<br/>
Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/RapidFireAI/rapidfireai">GitHub</a></i> ⭐
<br/>
To install RapidFire AI on your own machine, see the <a href="https://oss-docs.rapidfire.ai/en/latest/walkthrough.html">Install and Get Started</a> guide in our docs.
</div>

### RapidFire AI Tutorial Use Case: DPO for Alignment

In [ ]:
from rapidfireai import Experiment
from rapidfireai.fit.automl import List, RFGridSearch, RFModelConfig, RFLoraConfig, RFDPOConfig

### Load Dataset and Sample Train Subset

In [ ]:
from datasets import load_dataset

# Select a subset of the dataset for demo purposes
train_dataset = load_dataset(
    "trl-lib/ultrafeedback_binarized", 
    split="train").select(range(500))

### Initialize Experiment

In [ ]:
# Every experiment instance must be uniquely named
experiment = Experiment(experiment_name="exp1-sft-dpo", mode="fit")

#### Direct Preference Optimization (DPO) Training 

The first step is to train a **Supervised Fine-Tuning (SFT)** model to ensure the data we train on is in-distribution for the DPO algorithm.

Fine-tuning a language model via DPO consists of **two steps** and is significantly easier than PPO:

1. Data Collection: Gather a preference dataset containing:
    - **Positive examples**: High-quality generations given a prompt
    - **Negative examples**: Lower-quality generations for the same prompt
    - **Paired comparisons**: Each prompt has both chosen and rejected responses

2. Optimization: Maximize the log-likelihood of the DPO loss directly.

Since our goal is to demonstrate DPO, we will assume you already have **pretrained model or adapters from SFT training** and use them as our starting point.

### Model

##### Quantization and LoRA Config

We supervised fine-tuned (QLoRA) the `mistralai/Mistral-7B-Instruct-v0.3` base model by using preferred completions for supervision from the `trl-lib/ultrafeedback_binarized`. This ensures that the output paths sampled during the alignment training already exist in the model's distribution. We do not want the model to have to learn a new task from ground up during the alignment training. The goal of DPO training is to achieve alignment with human preferences by contrastively learning which outputs are considered preferred and which ones are dispreferred.

The quantization config for `rapidfire-ai/mistral-7b-sft-bnb-4bit` is below:
```python
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
```
We will use QLoRA to further fine-tune the SFT model for alignment with DPO

### Define Multi-Config Knobs for Model, LoRA, and DPO Trainer using RapidFire AI Wrapper APIs

We create a common base config, since most of the knobs will remain same across the configs in our experiment.

In [ ]:
from peft import TaskType

MODEL_NAME_OR_PATH = "rapidfire-ai-inc/mistral-7b-sft-bnb-4bit"

base_lora_config = RFLoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=64, 
    lora_alpha=64, 
    lora_dropout=0.1, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none", 
)

base_dpo_config = RFDPOConfig(
    model_adapter_name="default", # LoRA Adapter from original SFT trained model, if any. This adapter will be trained using DPO.
    ref_adapter_name="reference", # LoRA Adapter from original SFT trained model, if any. This adapter will remain unchanged and will be used for reference model.
    force_use_ref_model=False, 
    loss_type="sigmoid", # Uses Bradley-Terry model to calculate the loss.
    beta=0.1, 
    max_prompt_length=1024,
    max_completion_length=1024,
    max_length=2048, # Prompt + completion
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=5e-6,
    warmup_ratio=0.1,
    weight_decay=0,
    lr_scheduler_type="linear",
    num_train_epochs=1, 
    optim="adamw_8bit",
    bf16=True,
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=1
)

Now we create 4 different configs for experimentation by cloning the base config and only modifying the knobs we need.

In [ ]:
# Config 1 - Basic Bradley-Terry model with medium LoRA and large beta
lora_config_1 = base_lora_config.copy()
lora_config_1.r = 64
lora_config_1.lora_alpha = 64

dpo_config_1 = base_dpo_config.copy()
dpo_config_1.loss_type = "sigmoid"
dpo_config_1.beta = 0.1

# Config 2 - High Rank and encourage divergence from reference model
lora_config_2 = base_lora_config.copy()
lora_config_2.r = 128
lora_config_2.lora_alpha = 128

dpo_config_2 = base_dpo_config.copy()
dpo_config_2.loss_type = "sigmoid"
dpo_config_2.beta = 0.001

# Config 3 - Assumes noisy preference data, use label smoothing to handle it with Robust loss
lora_config_3 = base_lora_config.copy()
lora_config_3.r = 64
lora_config_3.lora_alpha = 64

dpo_config_3 = base_dpo_config.copy()
dpo_config_3.loss_type = "robust"
dpo_config_3.beta = 0.1
dpo_config_3.label_smoothing = 0.5

# Config 4 - Use a combined loss function with weighted sum
lora_config_4 = base_lora_config.copy()
lora_config_4.r = 64
lora_config_4.lora_alpha = 64

dpo_config_4 = base_dpo_config.copy()
dpo_config_4.loss_type = ["sigmoid", "bco_pair", "sft"]
dpo_config_4.loss_weights = [0.8, 0.2, 1.0]
dpo_config_4.beta = 0.1

In [ ]:
import torch

# List of 4 separate configs
config_set = List([
    RFModelConfig(
        model_name=MODEL_NAME_OR_PATH,
        ref_model_name=None,
        peft_config=lora_config_1,
        training_args=dpo_config_1,
        model_kwargs={ "device_map": "auto", "torch_dtype": torch.bfloat16},
        tokenizer_kwargs={"model_max_length": 2048, "padding_side": "right", "truncation": True}
    ),
    RFModelConfig(
        model_name=MODEL_NAME_OR_PATH,
        ref_model_name=None,
        peft_config=lora_config_2,
        training_args=dpo_config_2,
        model_kwargs={ "device_map": "auto", "torch_dtype": torch.bfloat16},
        tokenizer_kwargs={"model_max_length": 2048, "padding_side": "right", "truncation": True}
    ),
    RFModelConfig(
        model_name=MODEL_NAME_OR_PATH,
        ref_model_name=None,
        peft_config=lora_config_3,
        training_args=dpo_config_3,
        model_kwargs={ "device_map": "auto", "torch_dtype": torch.bfloat16},
        tokenizer_kwargs={"model_max_length": 2048, "padding_side": "right", "truncation": True}
    ),
    RFModelConfig(
        model_name=MODEL_NAME_OR_PATH,
        ref_model_name=None,
        peft_config=lora_config_4,
        training_args=dpo_config_4,
        model_kwargs={ "device_map": "auto", "torch_dtype": torch.bfloat16},
        tokenizer_kwargs={"model_max_length": 2048, "padding_side": "right", "truncation": True}
    )
])

#### Define Model Creation Function

In [ ]:
def sample_create_model(model_config): 
   """Function to create model object for any given config; must return tuple of (model, tokenizer)"""
   from transformers import AutoModelForCausalLM, AutoTokenizer

   model = AutoModelForCausalLM.from_pretrained(
      model_config["model_name"], 
      **model_config["model_kwargs"]
   )
   model.config.use_cache = False # Disable caching to save memory and stability during training

   tokenizer = AutoTokenizer.from_pretrained(
      model_config["model_name"], 
      **model_config["tokenizer_kwargs"]
   )
   if tokenizer.pad_token is None:
      tokenizer.pad_token = tokenizer.eos_token

   return (model, tokenizer)

#### Generate Config Group

In [ ]:
# Simple grid search across all sets of config knob values = 4 combinations in total
config_group = RFGridSearch(
    configs=config_set,
    trainer_type="DPO",
)

### Run Multi-Config Training

##### DPO Training Metrics

During training and evaluation, we track the following reward metrics:

| Metric | Description |
|--------|-------------|
| `rewards/chosen` | Mean difference between policy and reference model log probabilities for chosen responses (scaled by β) |
| `rewards/rejected` | Mean difference between policy and reference model log probabilities for rejected responses (scaled by β) |
| `rewards/accuracies` | Percentage of cases where chosen rewards > rejected rewards |
| `rewards/margins` | Mean difference between chosen and rejected rewards |

##### Mathematical Representation

- **Chosen Rewards**: $\beta * (\log \pi_\theta(y_w|x) - \log \pi_{ref}(y_w|x))$
- **Rejected Rewards**: $\beta * (\log \pi_\theta(y_l|x) - \log \pi_{ref}(y_l|x))$
- **Accuracy**: $\mathbb{E}[\mathbf{1}[r_{chosen} > r_{rejected}]]$
- **Margin**: $\mathbb{E}[r_{chosen} - r_{rejected}]$

Where:
- $\pi_\theta$ = policy model
- $\pi_{ref}$ = reference model  
- $y_w$ = chosen (winning) response
- $y_l$ = rejected (losing) response
- $\beta$ = temperature parameter

In [ ]:
# Launch training of all configs in the config_group with swap granularity of 4 chunks
experiment.run_fit(config_group, sample_create_model, train_dataset, eval_dataset=None, num_chunks=4, seed=42)

### End Current Experiment

In [ ]:
experiment.end()

<div align="center">
<a href="https://rapidfire.ai/"><img src="https://raw.githubusercontent.com/RapidFireAI/rapidfireai/main/docs/images/RapidFire - Blue bug -white text.svg" width="115"></a>
<a href="https://discord.gg/6vSTtncKNN"><img src="https://raw.githubusercontent.com/RapidFireAI/rapidfireai/main/docs/images/discord-button.svg" width="145"></a>
<a href="https://oss-docs.rapidfire.ai/"><img src="https://raw.githubusercontent.com/RapidFireAI/rapidfireai/main/docs/images/documentation-button.svg" width="125"></a>
<br/>
Thanks for trying RapidFire AI! ⭐ <i>Star us on <a href="https://github.com/RapidFireAI/rapidfireai">GitHub</a></i> ⭐
</div>
